In [1]:
import numpy as np 
import math 
import time 
import codecs 
from tqdm.autonotebook import tqdm

from collections import defaultdict

/home/chen/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


# 通用函数定义

In [2]:
def timmer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        stop_time = time.time()
        print("Func: {:s} | RunTime: {:f}".format(func.__name__, stop_time-start_time))
        return result 
    return wrapper

# 数据加载函数

In [3]:
# 查看分析数据
with open('data/hetrec2011-delicious-2k/bookmarks.dat', 'r', encoding="latin1") as f: 
    for i, line in enumerate(f):
        print(line)
        if i > 2: 
            break

id	md5	title	url	md5Principal	urlPrincipal

1	ab4954b633ddaf5b5bba6e9b71aa6b70	IFLA - The official website of the International Federation of Library Associations and Institutions	http://www.ifla.org/	7f431306c428457bc4e12b15634484f	www.ifla.org

2	2221e9cd106d269dd34682666f576fa3	gcdp-e.pdf (application/pdf Object)	http://archive.ifla.org/VII/s14/nd1/gcdp-e.pdf	1ef8cfcfe968101fa9b4e301847503d4	archive.ifla.org

7	c97c571dadaddbbb493126a0d4d01ba3	EdSelect	http://www.edselect.com/	792fd7eb20143386d0c4eb193c6124d	www.edselect.com



- 主要提取bookmark`id`以及`urlPrincipal`数据

In [4]:
with open("data/hetrec2011-delicious-2k/user_taggedbookmarks-timestamps.dat", 'r', encoding="latin1") as f: 
    for i, line in enumerate(f):
        print(line)
        if i>2: 
            break

userID	bookmarkID	tagID	timestamp

8	1	1	1289255362000

8	2	1	1289255159000

8	7	1	1289238901000



In [3]:
# 对数据进行处理
class Dataset:
    # 对每个用户按照时间进行从前到后的排序，取最后一个时间的item作为要预测的测试集
    def __init__(self, site=None):
        # site参数确认加载哪个网址
        self.bookmark_path = "data/hetrec2011-delicious-2k/bookmarks.dat"
        self.user_bookmark_path = "data/hetrec2011-delicious-2k/user_taggedbookmarks-timestamps.dat"
        self.site = site 
        self.loadData()
        
    def loadData(self):
        bookmarks = [f.strip() for f in codecs.open(self.bookmark_path, 'r', encoding="latin1").readlines()[1:]]
        site_ids = defaultdict(set)
        
        ## 获取网址所有的bookmarkID
        for b in bookmarks:
            #print(b)
            b = b.split("\t")
            site_ids[b[-1]].add(b[0])
        
        user_bookmarks = [f.strip() for f in codecs.open(self.user_bookmark_path, 'r', encoding="latin1").readlines()[1:]]
        data = defaultdict(set)
        cnt = 0 
        # 记录用户操作过的bookmarkID的用户ID以及对应的时间戳
        for ub in user_bookmarks:
            ub = ub.split("\t")
            if self.site is None or (self.site in site_ids and ub[1] in site_ids[self.site]):
                data[ub[0]].add((ub[1], int(ub[3][:-3])))
                cnt += 1 
        self.data = {k: list(sorted(list(data[k]), key=lambda x: x[1], reverse=True)) for k in data}
    
    def splitData(self):
        train, test = {}, {}
        for user in self.data: 
            if user not in train:
                train[user] = []
                test[user] = []
            data = self.data[user]
            train[user].extend(data[1:])
            test[user].append(data[0])
        
        return train, test
        

# 评价指标

In [4]:
class Metric:
    def __init__(self, train, test, GetRecommendation):
        self.train = train 
        self.test = test 
        self.GetRecommendation = GetRecommendation
        self.recs = self.getRec()
        
    # 为test中的每个用户推荐
    def getRec(self):
        recs = {}
        for user in self.test: 
            rank = self.GetRecommendation(user)
            recs[user] = rank
        return recs
    
    # 定义精确率指标计算方式
    def precision(self):
        all, hit = 0, 0 
        for user in self.test: 
            test_items = set([x[0] for x in self.test[user]])
            rank = self.recs[user]
            for item, score in rank:
                if item in test_items:
                    hit += 1 
            all += len(rank)
        return round(hit/all*100, 2) if all>0 else 0.0 
    
    # 定义召回率的计算方式
    def recall(self):
        all, hit = 0, 0 
        for user in self.test: 
            test_items = set([x[0] for x in self.test[user]])
            rank = self.recs[user]
            for item, score in rank:
                if item in test_items:
                    hit += 1 
            all += len(test_items)
        
        return round(hit/all*100, 2) if all>0 else 0.0 
    
    def eval(self):
        metric = {"Precision": self.precision(),
                 "Recall": self.recall()}
        
        return metric

# 算法实现

- RecentPopular
- TItemCF
- TUserCF
- ItemIUF
- UserIIF

In [5]:
# 1. 给用户推荐近期最热门物品
def RecentPopular(train, K, N, alpha=1.0, t0=int(time.time())):
    item_score = {}
    for user in train:
        for item, t in train[user]:
            if item not in item_score:
                item_score[item] = 0 
            item_score[item] += 1.0 / (alpha*(t0-t))
    
    item_score = list(sorted(item_score.items(), key=lambda x: x[1], reverse=True))
    
    def GetRecommendation(user):
        # 推荐N个最热门的未见过的
        user_items = set(train[user])
        rec_items = [x for x in item_score if x[0] not in user_items]
        return rec_items[:N]
    
    return GetRecommendation

In [14]:
# 2. 时间上下文相爱过关的ItemCF
def TItemCF(train, K, N, alpha=1.0, beta=1.0, t0=int(time.time())):
    ## 计算物品的的相似度矩阵
    sim = {}
    num = {}
    for user in train:
        items = train[user]
        for i in range(len(items)):
            u, t1 = items[i]
            ## 记录商品u被操作的次数
            num[u] = num.get(u, 0) + 1 
            if u not in sim:
                sim[u] = {}
            for j in range(len(items)):
                if j==i:
                    continue
                v, t2 = items[j]
                if u == v: continue
                if v not in sim[u]:
                    sim[u][v] = 0 
                sim[u][v] += 1.0 / (alpha * abs(t1-t2) + 1)
    for u in sim:
        for v in sim[u]:
            sim[u][v] /= math.sqrt(num[u]*num[v])
    
    ## 按照相似度排序
    sorted_item_sim = {k: list(sorted(v.items(), key=lambda x: x[1], reverse=True)) for k, v  in sim.items()}
    
    # 获取接口函数
    def GetRecommendation(user):
        items = {}
        seen_items = set(train[user])
        for item, t in train[user]:
            for u, _ in sorted_item_sim[item][:K]:
                if u not in seen_items:
                    if u not in items:
                        items[u] = 0 
                    items[u] += sim[item][u] / (1 + beta*(t0-t))
        recs = list(sorted(items.items(), key=lambda x: x[1], reverse=True))[:N]
        return recs 
    
    return GetRecommendation

In [20]:
# 3. 基于时间上下文的UserCF算法
def TUserCF(train, K, N, alpha=1.0, beta=1.0, t0=int(time.time())):
    ## 构建item-user的倒排索引
    item_users = defaultdict(list)
    for user in train: 
        for item, t in train[user]:
            item_users[item].append((user, t))
    
    # 计算用户的相似度矩阵
    sim = {}
    num = {}
    for item in item_users:
        users = item_users[item]
        for i in range(len(users)):
            u, t1 = users[i]
            num[u] = num.get(u, 0) + 1 
            
            if u not in sim:
                sim[u] = {}
            for j in range(len(users)):
                if j == i: continue
                v, t2 = users[j]
                if u == v: continue
                if v not in sim[u]:
                    sim[u][v] = 0 
                sim[u][v] += 1.0 / (alpha * abs(t1 - t2) + 1)
    for u in sim:
        for v in sim[u]:
            sim[u][v] /= math.sqrt(num[u] * num[v])
    
    # 按照相似度排序
    sorted_user_sim = {k: list(sorted(v.items(), key=lambda x: x[1], reverse=True)) for k, v in sim.items()}
    
    # 获取接口函数
    def GetRecommendation(user):
        items = {}
        seen_items = set(train[user])
        recs = []
        if user in sorted_user_sim:
            for u, _ in sorted_user_sim[user][:K]:
                for item, _ in train[u]:
                    if item not in seen_items:
                        if item not in items:
                            items[item] = 0 
                        items[item] += sim[user][u] / (1 + beta * (t0 - t))
            recs = list(sorted(items.items(), key=lambda x: x[1], reverse=True))[:N]
        return recs 
    
    return GetRecommendation

In [28]:
# 4.基于改进的物品余弦相似度推荐

def ItemIUF(train, K, N):
    # 计算物品的相似度矩阵
    sim = {}
    num = {}
    for user in train:
        items = train[user]
        for i in range(len(items)):
            u, _ = items[i]
            if u not in num:
                num[u] = 0 
            num[u] += 1 
            if u not in sim:
                sim[u] = {}
            for j in range(len(items)):
                if j==i: continue
                v, _  = items[j]
                if v == u: continue
                if v not in sim[u]:
                    sim[u][v] = 0 
                sim[u][v] += 1 / (math.log(1 + len(items)))
    
    for u in sim: 
        for v in sim[u]:
            sim[u][v] /= math.sqrt(num[u]*num[v])
    
    # 按照相似度排序
    sorted_item_sim = {k: list(sorted(v.items(), key=lambda x: x[1], reverse=True)) for k, v in sim.items()}
    
    def GetRecommendation(user):
        items = {}
        seen_items = set(train[user])
        for item, _ in train[user]:
            for u, _ in sorted_item_sim[item][:K]:
                if u not in seen_items:
                    if u not in items:
                        items[u] = 0 
                    items[u] += sim[item][u]
        recs = list(sorted(items.items(), key=lambda x: x[1], reverse=True))[:N]
        return recs
    
    return GetRecommendation
            

In [45]:
# 基于改进的余弦相似度的推荐
def UserIIF(train, K, N):
    # 计算item-user的倒排索引
    item_users = {}
    for user in train: 
        for item, _ in train[user]:
            if item not in item_users:
                item_users[item] = set()
            item_users[item].add(user)
    
    # 计算相似度矩阵
    sim = {}
    num = {}
    for item in item_users:
        users = list(item_users[item])
        for i in range(len(users)):
            u = users[i]
            if u not in num:
                num[u] = 0 
            num[u] += 1 
            if u not in sim:
                sim[u] = {}
            for j in range(len(users)):
                v = users[j]
                if v == u:
                    continue
                if v not in sim[u]:
                    sim[u][v] = 0 
                sim[u][v] += 1 / math.log(1 + len(users))
    for u in sim:
        for v in sim[u]:
            sim[u][v] /= math.sqrt(num[u]*num[v])
    
    sorted_user_sim = {k: list(sorted(v.items(), key=lambda x: x[1], reverse=True)) for k, v in sim.items()}
    
    # 获取接口函数
    def GetRecommendation(user):
        items = {}
        seen_items = set(train[user])
        recs = []
        if user in sorted_user_sim:
            for u, _ in sorted_user_sim[user][:K]:
                for item, _ in train[u]:
                    if item not in seen_items:
                        if item not in items:
                            items[item] = 0 
                        items[item] += sim[user][u]
            recs = list(sorted(items.items(), key=lambda x: x[1], reverse=True))[:N]
        return recs
    
    return GetRecommendation
    


# 实验

In [39]:
class Experiment:
    def __init__(self, K, N, site=None, rt="RecentPopular"):
        self.K = K 
        self.N = N 
        self.site = site
        self.rt = rt 
        self.alg = {"RecentPopular": RecentPopular, "TItemCF": TItemCF,
                   "TUserCF": TUserCF, "ItemIUF": ItemIUF, "UserIIF": UserIIF}
        
    # 定义单次实验
    def worker(self, train, test):
        getRecommendation = self.alg[self.rt](train, self.K, self.N)
        metric = Metric(train, test, getRecommendation)
        return metric.eval()
    
    
    # 运行多次实验
    @timmer
    def run(self):
        dataset = Dataset(self.site)
        train, test = dataset.splitData()
        metric = self.worker(train, test)
        print("Result (site={}, K={}, N={}): {}".format(self.site, self.K, self.N, metric))

## RecentPopular实验

In [11]:
K = 0 
for site in ["www.nytimes.com", "en.wikipedia.org"]:
    for N in range(10, 110, 10):
        exp = Experiment(K, N, site=site, rt="RecentPopular")
        exp.run()

Result (site=www.nytimes.com, K=0, N=10): {'Precision': 0.16, 'Recall': 1.58}
Func: run | RunTime: 0.472475
Result (site=www.nytimes.com, K=0, N=20): {'Precision': 0.11, 'Recall': 2.26}
Func: run | RunTime: 0.310248
Result (site=www.nytimes.com, K=0, N=30): {'Precision': 0.09, 'Recall': 2.71}
Func: run | RunTime: 0.339364
Result (site=www.nytimes.com, K=0, N=40): {'Precision': 0.08, 'Recall': 3.39}
Func: run | RunTime: 0.317156
Result (site=www.nytimes.com, K=0, N=50): {'Precision': 0.08, 'Recall': 3.84}
Func: run | RunTime: 0.333093
Result (site=www.nytimes.com, K=0, N=60): {'Precision': 0.1, 'Recall': 6.09}
Func: run | RunTime: 0.318316
Result (site=www.nytimes.com, K=0, N=70): {'Precision': 0.09, 'Recall': 6.09}
Func: run | RunTime: 0.343513
Result (site=www.nytimes.com, K=0, N=80): {'Precision': 0.08, 'Recall': 6.55}
Func: run | RunTime: 0.321679
Result (site=www.nytimes.com, K=0, N=90): {'Precision': 0.09, 'Recall': 7.67}
Func: run | RunTime: 0.334337
Result (site=www.nytimes.com,

## TItemCF实验

In [15]:
K = 10 
for site in ["www.nytimes.com", "en.wikipedia.org"]:
    for N in range(10, 110, 10):
        exp = Experiment(K, N, site=site, rt="TItemCF")
        exp.run()

Result (site=www.nytimes.com, K=10, N=10): {'Precision': 2.26, 'Recall': 2.26}
Func: run | RunTime: 0.329598
Result (site=www.nytimes.com, K=10, N=20): {'Precision': 2.14, 'Recall': 2.48}
Func: run | RunTime: 0.327818
Result (site=www.nytimes.com, K=10, N=30): {'Precision': 2.13, 'Recall': 2.48}
Func: run | RunTime: 0.304479
Result (site=www.nytimes.com, K=10, N=40): {'Precision': 2.13, 'Recall': 2.48}
Func: run | RunTime: 0.342988
Result (site=www.nytimes.com, K=10, N=50): {'Precision': 2.13, 'Recall': 2.48}
Func: run | RunTime: 0.311992
Result (site=www.nytimes.com, K=10, N=60): {'Precision': 2.13, 'Recall': 2.48}
Func: run | RunTime: 0.334342
Result (site=www.nytimes.com, K=10, N=70): {'Precision': 2.13, 'Recall': 2.48}
Func: run | RunTime: 0.302490
Result (site=www.nytimes.com, K=10, N=80): {'Precision': 2.13, 'Recall': 2.48}
Func: run | RunTime: 0.345126
Result (site=www.nytimes.com, K=10, N=90): {'Precision': 2.13, 'Recall': 2.48}
Func: run | RunTime: 0.303960
Result (site=www.ny

## TUserCF实验

In [21]:
K = 10 
for site in ["www.nytimes.com", "en.wikipedia.org"]:
    for N in range(10, 110, 10):
        exp = Experiment(K, N, site=site, rt="TUserCF")
        exp.run()

Result (site=www.nytimes.com, K=10, N=10): {'Precision': 3.36, 'Recall': 2.26}
Func: run | RunTime: 0.312162
Result (site=www.nytimes.com, K=10, N=20): {'Precision': 2.6, 'Recall': 2.26}
Func: run | RunTime: 0.322178
Result (site=www.nytimes.com, K=10, N=30): {'Precision': 2.72, 'Recall': 2.48}
Func: run | RunTime: 0.295560
Result (site=www.nytimes.com, K=10, N=40): {'Precision': 2.71, 'Recall': 2.48}
Func: run | RunTime: 0.347196
Result (site=www.nytimes.com, K=10, N=50): {'Precision': 2.71, 'Recall': 2.48}
Func: run | RunTime: 0.307859
Result (site=www.nytimes.com, K=10, N=60): {'Precision': 2.71, 'Recall': 2.48}
Func: run | RunTime: 0.302337
Result (site=www.nytimes.com, K=10, N=70): {'Precision': 2.71, 'Recall': 2.48}
Func: run | RunTime: 0.349459
Result (site=www.nytimes.com, K=10, N=80): {'Precision': 2.71, 'Recall': 2.48}
Func: run | RunTime: 0.302540
Result (site=www.nytimes.com, K=10, N=90): {'Precision': 2.71, 'Recall': 2.48}
Func: run | RunTime: 0.342327
Result (site=www.nyt

## ItemIUF

In [47]:
K = 10 
for site in ["www.nytimes.com", "en.wikipedia.org"]:
    for N in range(10, 110, 10):
        exp = Experiment(K, N, site=site, rt="ItemIUF")
        exp.run()

Result (site=www.nytimes.com, K=10, N=10): {'Precision': 2.26, 'Recall': 2.26}
Func: run | RunTime: 0.344931
Result (site=www.nytimes.com, K=10, N=20): {'Precision': 2.18, 'Recall': 2.48}
Func: run | RunTime: 0.303936
Result (site=www.nytimes.com, K=10, N=30): {'Precision': 2.15, 'Recall': 2.48}
Func: run | RunTime: 0.341212
Result (site=www.nytimes.com, K=10, N=40): {'Precision': 2.15, 'Recall': 2.48}
Func: run | RunTime: 0.318053
Result (site=www.nytimes.com, K=10, N=50): {'Precision': 2.15, 'Recall': 2.48}
Func: run | RunTime: 0.305131
Result (site=www.nytimes.com, K=10, N=60): {'Precision': 2.15, 'Recall': 2.48}
Func: run | RunTime: 0.336900
Result (site=www.nytimes.com, K=10, N=70): {'Precision': 2.15, 'Recall': 2.48}
Func: run | RunTime: 0.308511
Result (site=www.nytimes.com, K=10, N=80): {'Precision': 2.15, 'Recall': 2.48}
Func: run | RunTime: 0.443600
Result (site=www.nytimes.com, K=10, N=90): {'Precision': 2.15, 'Recall': 2.48}
Func: run | RunTime: 0.297744
Result (site=www.ny

## UserCF

In [48]:
K = 10 
for site in ["www.nytimes.com", "en.wikipedia.org"]:
    for N in range(10, 110, 10):
        exp = Experiment(K, N, site=site, rt="UserIIF")
        exp.run()

Result (site=www.nytimes.com, K=10, N=10): {'Precision': 3.69, 'Recall': 2.48}
Func: run | RunTime: 0.336690
Result (site=www.nytimes.com, K=10, N=20): {'Precision': 2.86, 'Recall': 2.48}
Func: run | RunTime: 0.292145
Result (site=www.nytimes.com, K=10, N=30): {'Precision': 2.72, 'Recall': 2.48}
Func: run | RunTime: 0.335682
Result (site=www.nytimes.com, K=10, N=40): {'Precision': 2.71, 'Recall': 2.48}
Func: run | RunTime: 0.304918
Result (site=www.nytimes.com, K=10, N=50): {'Precision': 2.71, 'Recall': 2.48}
Func: run | RunTime: 0.305903
Result (site=www.nytimes.com, K=10, N=60): {'Precision': 2.71, 'Recall': 2.48}
Func: run | RunTime: 0.339094
Result (site=www.nytimes.com, K=10, N=70): {'Precision': 2.71, 'Recall': 2.48}
Func: run | RunTime: 0.304954
Result (site=www.nytimes.com, K=10, N=80): {'Precision': 2.71, 'Recall': 2.48}
Func: run | RunTime: 0.359146
Result (site=www.nytimes.com, K=10, N=90): {'Precision': 2.71, 'Recall': 2.48}
Func: run | RunTime: 0.311465
Result (site=www.ny